In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import BlockRNNModel
from darts.metrics import smape
from darts.dataprocessing.transformers import Scaler
import torch
from pytorch_lightning.callbacks import Callback, EarlyStopping
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Configurações de visualização
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

In [2]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

DATA_FILE_PATH = "../data/ground_station_clean.csv"
df = pd.read_csv(DATA_FILE_PATH)

In [3]:
encoders = {
    "cyclic": {
        "past": ["month", "dayofyear", "day", "hour", "minute"],
        "future": ["month", "dayofyear", "day", "hour", "minute"]
    },
    "transformer": Scaler(),
    "datetime_attribute": {
        "past": ["year"],
        "future": ["year"]
    }
}


In [4]:
class PatchedPruningCallback(PyTorchLightningPruningCallback, Callback):
    pass

# Create directory to save iteration results
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [5]:
from sklearn.model_selection import TimeSeriesSplit

def objective(trial):
    input_chunk_length = trial.suggest_int("input_chunk_length", 24, 168, step=12)
    hidden_size = trial.suggest_int("hidden_size", 32, 256, step=8)
    layer_size = trial.suggest_int("layer_size", 1, 3)
    dropout = trial.suggest_float("dropout", 0.1, 0.5, step=0.05)
    batch_size = trial.suggest_int("batch_size", 64, 256, step=8)
    lr = trial.suggest_float("lr", 1e-6, 5e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)

    early_stopper = EarlyStopping("val_loss", min_delta=0.0005, patience=15, verbose=True)
    prunner = PatchedPruningCallback(trial, monitor="val_loss")
    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": [early_stopper, prunner],
    }

    n_splits = 3
    tscv = TimeSeriesSplit(n_splits=n_splits)
    overall_smape_list = []
    fold = 0

    for train_index, val_index in tscv.split(df):
        print(f"\nFold {fold+1}/{n_splits}")
        train_df_fold = df.iloc[train_index]
        val_df_fold = df.iloc[val_index]
        train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
        val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

        scaler = Scaler()
        scaler = scaler.fit(train_fold)
        train_scaled = scaler.transform(train_fold)
        val_scaled = scaler.transform(val_fold)

        opt_kwargs = {"lr": lr, "weight_decay": weight_decay}

        model = BlockRNNModel(
            model="LSTM",
            model_name=f"model_{fold+1}",
            work_dir="/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs",
            input_chunk_length=input_chunk_length,
            output_chunk_length=24,
            hidden_dim=hidden_size,
            n_rnn_layers=layer_size,
            n_epochs=30,
            batch_size=batch_size,
            dropout=dropout,
            add_encoders=encoders,
            pl_trainer_kwargs=pl_trainer_kwargs,
            loss_fn=torch.nn.L1Loss(),
            optimizer_cls=torch.optim.Adam,
            lr_scheduler_cls=torch.optim.lr_scheduler.ReduceLROnPlateau,
            lr_scheduler_kwargs={"mode": "min", "factor": 0.3, "patience": 7, "min_lr": 1e-7},
            save_checkpoints=True,
            force_reset=True,
            random_state=42,
            optimizer_kwargs=opt_kwargs,
        )
        model.fit(
            series=train_scaled,
            val_series=val_scaled,
            verbose=False,
            dataloader_kwargs={"num_workers": 11},
        )

        try:
            loaded_model = model.load_from_checkpoint(f"/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs/model_{fold+1}", best=True)
            print(f"Model loaded from checkpoint for trial {trial.number}, fold {fold}")
        except FileNotFoundError:
            print(f"Checkpoint not found. Using the in-memory trained model.")
            loaded_model = model

        forecasts = loaded_model.historical_forecasts(
            val_scaled,
            forecast_horizon=24,
            stride=1,
            retrain=False,
            verbose=False
        )

        overall_smape_val = smape(val_scaled, forecasts)
        print(f"SMAPE fold {fold}: {overall_smape_val}")
        overall_smape_list.append(overall_smape_val)
        fold += 1

    mean_smape = np.mean(overall_smape_list)
    print(f"Média dos SMAPE nos folds: {mean_smape}")

    trial_dict = {
        "trial_number": trial.number,
        "fold_smape": overall_smape_list,
        "mean_smape": mean_smape,
        "hyperparameters": trial.params
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Results of trial {trial.number} saved in {json_path}")
    return mean_smape if not np.isnan(mean_smape) else float("inf")

In [6]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [14]:
study = optuna.create_study(direction="minimize")
num_hyperparams = 7
n_trials = 7 * num_hyperparams
study.optimize(objective, n_trials=n_trials, callbacks=[print_callback])

[I 2025-06-09 18:46:41,096] A new study created in memory with name: no-name-6e8454ee-59ca-4df2-9a35-f11d31df5d6f
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.30000000000000004 and num_layers=1
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Fold 1/3


Metric val_loss improved. New best score: 8.293
Metric val_loss improved by 0.019 >= min_delta = 0.0005. New best score: 8.274
Metric val_loss improved by 0.023 >= min_delta = 0.0005. New best score: 8.252
Metric val_loss improved by 0.015 >= min_delta = 0.0005. New best score: 8.237
Metric val_loss improved by 0.003 >= min_delta = 0.0005. New best score: 8.234
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 8.233
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 8.231
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 8.230
Metric val_loss improved by 0.003 >= min_delta = 0.0005. New best score: 8.227
Metric val_loss improved by 0.003 >= min_delta = 0.0005. New best score: 8.224
Metric val_loss improved by 0.015 >= min_delta = 0.0005. New best score: 8.209
Metric val_loss improved by 0.007 >= min_delta = 0.0005. New best score: 8.202
Metric val_loss improved by 0.004 >= min_delta = 0.0005. New best score: 8.198
Metr

Model loaded from checkpoint for trial 0, fold 0


Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.30000000000000004 and num_layers=1
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

SMAPE fold 2: 71.4172058817696
Média dos SMAPE nos folds: 72.11611886981088
Results of trial 46 saved in optuna_iteration_metrics/trial_46.json
Current value: 72.11611886981088, Current params: {'input_chunk_length': 36, 'hidden_size': 48, 'layer_size': 1, 'dropout': 0.4, 'batch_size': 152, 'lr': 0.00035816530162297017, 'weight_decay': 0.0011389404400731398}
Best value: 68.0697360044159, Best params: {'input_chunk_length': 60, 'hidden_size': 152, 'layer_size': 1, 'dropout': 0.1, 'batch_size': 168, 'lr': 0.00044033332793109764, 'weight_decay': 0.0008873088827014805}

Fold 1/3


[I 2025-06-09 20:14:33,883] Trial 47 pruned. Trial was pruned at epoch 0.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Current value: 8.060837787265879, Current params: {'input_chunk_length': 84, 'hidden_size': 120, 'layer_size': 2, 'dropout': 0.35, 'batch_size': 128, 'lr': 0.0001145857649584245, 'weight_decay': 0.007956473371941292}
Best value: 68.0697360044159, Best params: {'input_chunk_length': 60, 'hidden_size': 152, 'layer_size': 1, 'dropout': 0.1, 'batch_size': 168, 'lr': 0.00044033332793109764, 'weight_decay': 0.0008873088827014805}

Fold 1/3


[I 2025-06-09 20:14:37,987] Trial 48 pruned. Trial was pruned at epoch 0.


Current value: 8.160107169522004, Current params: {'input_chunk_length': 96, 'hidden_size': 240, 'layer_size': 1, 'dropout': 0.1, 'batch_size': 96, 'lr': 5.412316721248238e-05, 'weight_decay': 6.595189512106656e-05}
Best value: 68.0697360044159, Best params: {'input_chunk_length': 60, 'hidden_size': 152, 'layer_size': 1, 'dropout': 0.1, 'batch_size': 168, 'lr': 0.00044033332793109764, 'weight_decay': 0.0008873088827014805}


In [15]:
print(f"Best MASE Value (Minimum): {study.best_value}")
print(f"Best Parameters: {study.best_params}")

Best MASE Value (Minimum): 68.0697360044159
Best Parameters: {'input_chunk_length': 60, 'hidden_size': 152, 'layer_size': 1, 'dropout': 0.1, 'batch_size': 168, 'lr': 0.00044033332793109764, 'weight_decay': 0.0008873088827014805}


In [9]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Best results saved in {json_path}")


Melhores resultados salvos em optuna_iteration_metrics/best_trial.json
